In [1]:
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [2]:
data = pd.read_csv("data.csv")

In [3]:
MAP = {
    "r1":["r2","r3"],
    "r2":["r1","r4"],
    "r3":["r1","r7"],
    "r4":["r2","r8"],
    "r5":["r6","r9","c3"],
    "r6":["r5","c3"],
    "r7":["r3","c1"],
    "r8":["r4","r9"],
    "r9":["r5","r8","r13"],
    "r10":["c3"],
    "r11":["c3"],
    "r12":["outside","r22"],
    "r13":["r9","r24"],
    "r14":["r24"],
    "r15":["c3"],
    "r16":["c3"],
    "r17":["c3"],
    "r18":["c3"],
    "r19":["c3"],
    "r20":["c3"],
    "r21":["c3"],
    "r22":["r12","r25"],
    "r23":["r24"],
    "r24":["r13","r14","r23"],
    "r25":["r22","r26","c1"],
    "r26":["r25","r27"],
    "r27":["r26","r32"],
    "r28":["c4"],
    "r29":["c4","r30"],
    "r30":["r29"],
    "r31":["r32"],
    "r32":["r27","r31","r33"],
    "r33":["r32"],
    "r34":["c2"],
    "r35":["c4"],
    "c1":["r7","r25","c2"],
    "c2":["c1","r34","c4"],
    "c3": ["r5","r6","r10","r11","r15","r16","r17","r18","r19","r20","r21","o1"],
    "c4":["r29","c2","r35","r28","o1"],
    "o1":["c3","c4"],
    "outside":["r12"]  
}
censor_DF =  {
    "r1": "us3", #us for unreliable_sensor
    "r5": "rs2" ,#rs for reliable_sensor
    "r8": "ds1", #ds for door sensor
    "r9": "ds1",
    "r16": "rs1",
    "r24" : "us4",
    "r25" : "rs3",
    "r26" : "ds3", 
    "r27" : "ds3",
    "r31" : "rs4",
    "r35" : "ds4",
    "c1" : "ds2",
    "c2" : "ds2",
    "c3" : "us2",
    "c4" : "ds4",
    "o1" : "us1"
}

In [10]:
index1 = data[data.time == "08:01:00"].index.tolist()[0] #  get in entrance ends
index2 = data[data.time == "08:05:00"].index.tolist()[0] # People tends to be stable
index3 = data[data.time=="17:30:00"].index.tolist()[0] # leave work starts
index4 = data[data.time=="17:41:00"].index.tolist()[0] # leave work ends

In [11]:
data1 = data.iloc[0:index1] # 08:00 - 08:01
data2 = data.iloc[index1:index2].reset_index(drop=True) #08:01 - 08:05
data3 = data.iloc[index2:index3].reset_index(drop=True) # 08:05 - 17:30
data4 = data.iloc[index3:index4].reset_index(drop=True) # 17:30 - 17:41
data5 = data.iloc[index4:].reset_index(drop=True)
data_lst = [data1,data2,data3,data4,data5]

In [12]:
# self_trans means the probability of a person keeps staying in the same area
self_trans = {key:[] for key in MAP.keys()} 

for sub_data in data_lst:
    for place in MAP.keys():
        col_data = sub_data[place] 
        stay = 0
        go = 0
        for i in range(1,col_data.shape[0]):
            if(col_data[i]>=col_data[i-1]):
                stay += col_data[i-1]
            else:
                stay += col_data[i]
                go += col_data[i-1] - col_data[i]
        total = stay+go
        if(total!=0):
            self_trans[place].append(stay/total)
        else:
            self_trans[place].append(0)
                    
            
print(self_trans)

{'r1': [0, 1.0, 0.9260340632603407, 0.3333333333333333, 0], 'r2': [0, 0, 0.6417910447761194, 0.0, 0], 'r3': [0, 0.9, 0.9156235512285582, 0.3333333333333333, 0], 'r4': [0, 0, 0.8225602027883396, 0.5, 0], 'r5': [0, 0, 0.6028169014084507, 0.0, 0], 'r6': [0, 0, 0.9585492227979274, 0, 0], 'r7': [0, 0.0, 0.9445903361344538, 0.5, 0], 'r8': [0, 0, 0.2654320987654321, 0, 0], 'r9': [0, 0, 0.8112058465286236, 0, 0], 'r10': [0, 1.0, 0.985493898227032, 0, 0], 'r11': [0, 1.0, 0.9837587006960556, 0.5, 0], 'r12': [0.8888888888888888, 0, 0.9613259668508287, 0.6, 0], 'r13': [0, 0, 0.8971848225214198, 0, 0], 'r14': [0, 0, 0.9796251018744906, 0, 0], 'r15': [0, 1.0, 0.9862700228832952, 0, 0], 'r16': [0, 0.7142857142857143, 0.9833518312985572, 0.9354838709677419, 0], 'r17': [0, 0.6666666666666666, 0.972939729397294, 0, 0], 'r18': [0, 1.0, 0.9826539462272333, 0, 0], 'r19': [0, 0.5, 0.979154603358425, 0.5, 0], 'r20': [0, 0, 0.9761904761904762, 0, 0], 'r21': [0, 0, 0.9595687331536388, 0, 0], 'r22': [1.0, 0.166

## Find Neighbour Transition

In [13]:
def n_step_neighbour(node,n,G):
    neighbour_list = []
    neighbour_list = neighbour_list + G[node]
    while n > 1:
        for new_node in neighbour_list:
            neighbour_list = neighbour_list+ G[new_node]
        n = n -1 
    n_list = list(set(neighbour_list))
    n_list.remove(node)
    return(n_list)

In [14]:
neighbour_MAP = {key:n_step_neighbour(key,2,MAP) for key in MAP.keys()}

In [15]:
neighbour_MAP

{'r1': ['r4', 'r2', 'r3', 'r7'],
 'r2': ['r4', 'r3', 'r8', 'r1'],
 'r3': ['r2', 'c1', 'r1', 'r7'],
 'r4': ['r2', 'r9', 'r8', 'r1'],
 'r5': ['r18',
  'r19',
  'r21',
  'r10',
  'r6',
  'r15',
  'c3',
  'r11',
  'r16',
  'o1',
  'r17',
  'r9',
  'r8',
  'r13',
  'r20'],
 'r6': ['r18',
  'r19',
  'r21',
  'r10',
  'r15',
  'c3',
  'r11',
  'r16',
  'o1',
  'r17',
  'r9',
  'r5',
  'r20'],
 'r7': ['r3', 'r25', 'c1', 'c2', 'r1'],
 'r8': ['r4', 'r2', 'r9', 'r13', 'r5'],
 'r9': ['r4', 'r6', 'c3', 'r8', 'r13', 'r24', 'r5'],
 'r10': ['r18',
  'r19',
  'r21',
  'r6',
  'r15',
  'c3',
  'r11',
  'r16',
  'o1',
  'r17',
  'r5',
  'r20'],
 'r11': ['r18',
  'r19',
  'r21',
  'r10',
  'r6',
  'r15',
  'c3',
  'r16',
  'o1',
  'r17',
  'r5',
  'r20'],
 'r12': ['r25', 'r22', 'outside'],
 'r13': ['r14', 'r23', 'r9', 'r8', 'r24', 'r5'],
 'r14': ['r13', 'r23', 'r24'],
 'r15': ['r18',
  'r19',
  'r21',
  'r10',
  'r6',
  'c3',
  'r11',
  'r16',
  'o1',
  'r17',
  'r5',
  'r20'],
 'r16': ['r18',
  'r19',
  

In [23]:
place_data = data.iloc[:,17:58]
pl_diff = place_data.diff()
# place_data1 = place_data.iloc[0:index1]
# place_data2 = place_data.iloc[index1:index2].reset_index(drop=True)
# place_data3 = place_data.iloc[index2:index3].reset_index(drop=True)
# place_data4 = place_data.iloc[index3:index4].reset_index(drop=True)
# place_data5 = place_data.iloc[index4:].reset_index(drop=True)
# place_data_lst = [place_data1,place_data2,place_data3,place_data4,place_data5]

In [24]:
pl_diff

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,...,r32,r33,r34,r35,c1,c2,c3,c4,o1,outside
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# neigh_trans means the probability of a person transfers to the neighbourhood
neigh_trans = {key:{} for key in MAP.keys()} 
for key,neighbours in neighbour_MAP.items():
    for neighbour in neighbours:
        neigh_trans[key][neighbour]=[]

In [12]:
neigh_trans

{'r1': {'r7': [], 'r4': [], 'r2': [], 'r3': []},
 'r2': {'r1': [], 'r4': [], 'r3': [], 'r8': []},
 'r3': {'r1': [], 'r7': [], 'c1': [], 'r2': []},
 'r4': {'r1': [], 'r9': [], 'r2': [], 'r8': []},
 'r5': {'r10': [],
  'r18': [],
  'r16': [],
  'r20': [],
  'r17': [],
  'r6': [],
  'r9': [],
  'r11': [],
  'o1': [],
  'r13': [],
  'r21': [],
  'r19': [],
  'c3': [],
  'r8': [],
  'r15': []},
 'r6': {'r5': [],
  'r10': [],
  'r18': [],
  'r16': [],
  'r20': [],
  'r17': [],
  'r9': [],
  'r11': [],
  'o1': [],
  'r21': [],
  'r19': [],
  'c3': [],
  'r15': []},
 'r7': {'c2': [], 'r1': [], 'c1': [], 'r3': [], 'r25': []},
 'r8': {'r5': [], 'r4': [], 'r2': [], 'r9': [], 'r13': []},
 'r9': {'r5': [],
  'r24': [],
  'r6': [],
  'r4': [],
  'r13': [],
  'c3': [],
  'r8': []},
 'r10': {'r5': [],
  'r18': [],
  'r16': [],
  'r20': [],
  'r17': [],
  'r6': [],
  'r11': [],
  'o1': [],
  'r21': [],
  'r19': [],
  'c3': [],
  'r15': []},
 'r11': {'r5': [],
  'r10': [],
  'r18': [],
  'r16': [],
  'r

In [ ]:
for sub_data in data_lst:
    place_data = sub_data.iloc[:,17:58]
    for place in MAP.keys():
        
            
        
        

print(self_trans)